
# DTW

In [1]:
# import thư viện cần sử dụng
import os
import re
import librosa
import numpy as np
from tqdm import tqdm


# Đọc, xử lý dữ liệu, tách đặc trưng


In [2]:
def normalize(feature):
    normalized = np.full_like(feature, 0)
    for i in range(feature.shape[1]):
        normalized[:,i] = feature[:,i] - np.mean(feature[:,i]) # đưa trung bình về 0
        normalized[:,i] = normalized[:,i] / np.max(np.abs(normalized[:,i])) # đưa khoảng giá trị về [-1, 1]
    return normalized

In [3]:
def mfcc(path):
    #load file bằng thư viện librosa
    audio = librosa.load(path)[0]
    #trích xuất lần lượt mfcc ,delta và deltadelta
    mfcc = librosa.feature.mfcc(y=audio, n_mfcc=13)
    delta = librosa.feature.delta(mfcc)
    deltadelta = librosa.feature.delta(mfcc, order=2)
    
    return normalize(np.concatenate((mfcc, delta, deltadelta)))

In [5]:
def build_average_template(command, metric='euclidean'): # command = a b len xuong trai phai ban
    path = 'D:\\xltn\\dataset\\'+command+"\\"
    p_train = [path+i for i in np.random.choice(os.listdir(path),3)]
    
    templates = [mfcc(p) for p in p_train]
    #print(type(templates[0]))
    
    _, align01 = librosa.sequence.dtw(X=templates[0], Y=templates[1])
    _, align02 = librosa.sequence.dtw(X=templates[0], Y=templates[2])
    count = np.ones(len(templates[0][0])) # count[i] đếm số vectơ được dóng vào vị trí i của template chuẩn
    sum = templates[0].copy() # tổng của 3 template sau khi dóng

    for t, q in align01:
        count[t] += 1
        sum[:,t] += templates[1][:,q]

    for t, q in align02:
        count[t] += 1
        sum[:,t] += templates[2][:,q]

    average_template = sum / count

    return average_template

In [6]:
commands = ['a', 'b', 'len', 'xuong', 'trai', 'phai', 'ban', 'nhay']
model_reconizes = [build_average_template(cmd) for cmd in commands]

In [7]:
def predict_lable(path):
    template_predict = mfcc(path)
    costs = []
    for model in model_reconizes:
        cost = librosa.sequence.dtw(template_predict, model)[0][-1,-1]
        costs.append(cost)
    return commands[np.argmin(costs)]

In [12]:
def get_test_path():
    input_folder = 'D:\\xltn\\dataset\\'
    test_paths = []
    for path in os.listdir(input_folder):
        label = path
        arr = os.listdir(input_folder+path)
        test_arr = np.random.choice(arr,50)
        for test_file in test_arr:
            test_paths.append({
                "test_path" : input_folder+path+"\\"+test_file,
                "label": label})
    return test_paths
def test():
    test_paths = get_test_path()
    total = 0
    predict_true = 0
    predict_false = {'a':0, 'b':0, 'len':0, 'xuong':0, 'trai':0, 'phai':0, 'ban':0, 'nhay':0}
    for test in tqdm(test_paths):
        try:
            predict_label = predict_lable(test["test_path"])
            if predict_label == test["label"]:
                predict_true +=1
            else:
                predict_false[test["label"]]+=1
            total +=1
        except:
            pass
    print('độ chính xác: {:.2f}%\n'.format(predict_true/total*100))
    print("Số lần nhận diện sai của từng nhãn:")
    print(predict_false)

In [13]:
test()

100%|████████████████████████████████████████████████████████████████████████████████| 400/400 [00:11<00:00, 34.74it/s]

độ chính xác: 52.50%

Số lần nhận diện sai của từng nhãn:
{'a': 18, 'b': 47, 'len': 25, 'xuong': 16, 'trai': 35, 'phai': 12, 'ban': 29, 'nhay': 8}
